In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
import os
import matplotlib.pyplot as plt
import pickle

from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans

fase = '02b_DTW'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_norm = gen_config['path_norm']
path_dtw = gen_config['path_dtw']
clusters = gen_config['var_clusters']
corte_prod_dtw = gen_config['var_corte_prod_dtw']
leer_pickle_dtw = gen_config['var_leer_pickle_dtw']
path_traindtw = gen_config['path_traindtw']
path_fitdtw = gen_config['path_fitdtw']
path_dtwmodel = gen_config['path_dtw_model']
ejecutar_dtw = gen_config['var_ejecutar_dtw']
dibujar_dtw = gen_config['var_dibujar_dtw']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
----------------------------------------------02b_DTW-----------------------------------------------


In [4]:
df_norm = pd.read_parquet(f"{folder}/{path_norm}")

In [5]:
df_norm.sort_values(by=['product_id', 'customer_id','periodo'], inplace=True)
df_train = df_norm[df_norm['product_id'] <= corte_prod_dtw]
df_fit = df_norm[df_norm['product_id'] > corte_prod_dtw]

print(f"df_norm shape:   ({df_norm.shape[0]:>9_d},{df_norm.shape[1]:_d})")
print(f"df_train shape:  ({df_train.shape[0]:>9_d},{df_train.shape[1]:_d})")
print(f"df_fit shape:    ({df_fit.shape[0]:>9_d},{df_fit.shape[1]:_d})")

df_norm shape:   (2_293_481,15)
df_train shape:  (  453_297,15)
df_fit shape:    (1_840_184,15)


In [6]:
prodcust_train = df_train[['product_id', 'customer_id']].drop_duplicates()
prodcust_fit = df_fit[['product_id', 'customer_id']].drop_duplicates()

print(f"prodcust_train series:  ({prodcust_train.shape[0]:>9_d},{df_train.shape[1]:_d})")
print(f"prodcust_fit series:    ({prodcust_fit.shape[0]:>9_d},{df_fit.shape[1]:_d})")

prodcust_train series:  (   37_722,15)
prodcust_fit series:    (  225_083,15)


In [7]:
if leer_pickle_dtw:
    X_train = np.load(f"{folder}/{path_traindtw}")
    print(f"series_train shape: {X_train.shape}")
    
else:
    series_train = []
    i = 0
    for producto, cliente in prodcust_train.values:
        i +=1
        print(f"Timeseries {i} de {prodcust_train.shape[0]}: {producto}, {cliente}", end="\r")
        series_train.append(df_train[(df_train["product_id"] == producto) & (df_train["customer_id"] == cliente)][['tn_norm']])
        X_train = to_time_series_dataset(series_train)
        np.save(f"{folder}/{path_traindtw}", X_train)
    # with open('series_train.pickle', 'wb') as handle:
    #     pickle.dump(series_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

series_train shape: (37722, 36, 1)


In [8]:
if leer_pickle_dtw:
    X_fit = np.load(f"{folder}/{path_fitdtw}")
    print(f"series_fit shape: {X_fit.shape}")
    
else:
    series_fit = []
    i = 0
    for producto, cliente in prodcust_fit.values:
        i +=1
        print(f"Timeseries {i} de {prodcust_fit.shape[0]}: {producto}, {cliente}", end="\r")
        series_fit.append(df_fit[(df_fit["product_id"] == producto) & (df_fit["customer_id"] == cliente)][['tn_norm']])
    
    X_fit = to_time_series_dataset(series_fit)
    np.save(f"{folder}/{path_fitdtw}", X_fit)
    # with open('series_fit.pickle', 'wb') as handle:
    #     pickle.dump(series_fit, handle, protocol=pickle.HIGHEST_PROTOCOL)

series_fit shape: (225083, 36, 1)


In [13]:
np.nan_to_num(X_train, nan=0, copy=False)
print(X_train.shape)

(37722, 36, 1)


In [15]:
if ejecutar_dtw:
    if leer_pickle_dtw:
        x_clusters_dtw = np.full((X_train.shape[0], len(clusters)), np.nan)
        model_dtw = []
        model_dtw.append(TimeSeriesKMeans.from_pickle(f"{folder}/{path_dtwmodel}"))
    else:
        x_clusters_dtw = np.full((X_train.shape[0], len(clusters)), np.nan)
        model_dtw = []

        for i in range(len(clusters)):
            start_time = datetime.now()
            model = TimeSeriesKMeans(n_clusters=clusters[i], metric="dtw",
                                    max_iter=100, random_state=42, max_iter_barycenter=100,
                                    n_jobs = -1, verbose = 0)
            x_clusters_dtw[:,i] = model.fit_predict(X_train)
            print(f"DTW Corrida {i}, clusters: {model.n_clusters}, inertia: {model.inertia_:.2f}, time: {(datetime.now()-start_time).total_seconds():.2f}")
            model_dtw.append(model)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 12744 task

4.161 --> 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 12744 task

3.438 --> 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 12744 task

...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 12744 task

3.211 --> 


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 12744 task

DTW Corrida 0, clusters: 10, inertia: 3.21, time: 4291.09


[Parallel(n_jobs=-1)]: Done 377220 out of 377220 | elapsed:  1.7min finished


In [16]:
path_dtw_model = f"02b_dtw_model.pkl"
model_dtw[0].to_pickle(f"{folder}/{path_dtw_model}")

In [35]:
if ejecutar_dtw:
    df_clusters = pd.concat([prodcust_train.reset_index(drop=True), pd.DataFrame(x_clusters_dtw)], axis=1)
    cols = ['product_id', 'customer_id']
    for method in ['cluster_dtw']:
        for cluster in clusters:
            cols.append(f"{method}_{cluster:0>2}")

    df_clusters.columns = cols

In [38]:
np.nan_to_num(X_fit, nan=0, copy=False)
print(X_fit.shape)

(225083, 36, 1)


In [39]:
if ejecutar_dtw:
    x_clusters_dtw_fit = np.full((X_fit.shape[0], len(clusters)), np.nan)
    x_clusters_dtw_fit[:,0] = model_dtw[0].predict(X_fit)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 394 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 2394 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 3144 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 3994 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 4944 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 5994 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 7144 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 8394 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 9744 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 11194 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 12744 task

In [41]:
if ejecutar_dtw:
    df_clusters_fit = pd.concat([prodcust_fit.reset_index(drop=True), pd.DataFrame(x_clusters_dtw_fit)], axis=1)
    cols = ['product_id', 'customer_id']
    for method in ['cluster_dtw']:
        for cluster in clusters:
            cols.append(f"{method}_{cluster:0>2}")

    df_clusters_fit.columns = cols

In [43]:
if ejecutar_dtw:
    df_clusters_full = pd.concat([df_clusters, df_clusters_fit], axis=0)
    df_clusters_full.to_parquet(f"{folder}/{path_dtw}", index=False)

In [44]:
if dibujar_dtw:
    fig, axs = plt.subplots(max(clusters), len(clusters), figsize=(24, 40))

    for i in range(len(clusters)):
        y_pred = x_clusters_dtw[:,i]

        for j in range(clusters[i]):
            for xx in X_train[y_pred == j]:
                axs[j, i].plot(xx.ravel(), "k-", alpha=.1)
            axs[j, i].plot(model_dtw[i].cluster_centers_[j].ravel(), color='green')
            axs[j, i].set_title(f"Cluster {j} de {clusters[i]} DTW")
            
            # for xx in X_train[y_pred == j]:
            #     axs[j, (i+4)].plot(xx.ravel(), "k-", alpha=.1)
            # axs[j, (i+4)].plot(model_softdtw[i].cluster_centers_[j].ravel(), color='blue')
            # axs[j, (i+4)].set_title(f"Cluster {j} de {clusters[i]} Soft_DTW")

    plt.tight_layout()

    plt.show()

In [45]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

----------------------------------------------02b_DTW-----------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



